In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier

In [38]:
nuclear = pd.read_csv("C:/Users/natha/Desktop/MSCi/nucleardata.csv")

In [39]:
nuclear.head()

,Transcript name,5UTR,3UTR,Mettler C0,Tertile
0,ADC1,ATAAAGGTCGGGCCGGTTGTTTACAATGAAGGGCACAGCCGCGCCA...,AGTGGACTGTCGGCATAAGAATGCTGTGCTAGGAATAGGTACCGTG...,9.527054,2
1,LHCSR1,AGAGTCATTCCCCAACCCACTTCTCAGCTTGCACCTCCTTGTGCAC...,GTGCCTCTTCCTCGATCGTGATCTGCTTCACGGTTTGGTCGCGGGC...,12.158357,3
2,GRX2,CACATTGGCACTGGAGTTTATTTGAGGCGATACATGCCTTCTCATG...,GGCGAGGGCGGCGGGGTGGGCAGCAGGTTGTGAGGCGCGTGCAGGG...,14.064211,3
3,TIM16,GCACCAAACTTTCCCTGGGCCCCCCTCTGGGCGCACTCGAAATGCT...,CGAGGCGTGGCGTGAGGTGACAAGCAGGGGCGATGGATGTTGGGGA...,8.464260,2
4,Cre01.g056200,ACTTGTCAAAACCTTGCAAGGCTTGCGGCAGGCACTGTCCAGTACC...,TGGGTCAGTGGGGCGTGTGTGTGGCCGGCTGCTGGACCCGTGAGCA...,14.582071,3


In [40]:
nuclear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Transcript name  384 non-null    object 
 1   5UTR             384 non-null    object 
 2   3UTR             384 non-null    object 
 3   Mettler C0       384 non-null    float64
 4   Tertile          384 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 15.1+ KB


In [41]:
nuclear.keys()

Index(['Transcript name', '5UTR', '3UTR', 'Mettler C0', 'Tertile'], dtype='object')

In [42]:
def kmerconvert (sequence, k_length):
    k_conversion = []
    num_kmers = len(sequence) - k_length + 1
    
    for element in range(num_kmers):
        kmer = sequence[element:element + k_length]
        k_conversion.append(kmer)
        
    return k_conversion

In [43]:
nuclear['3UTRK'] = nuclear.apply(lambda x: kmerconvert(x['3UTR'], k_length=3), axis=1)

In [44]:
nuclear['5UTRK'] = nuclear.apply(lambda x: kmerconvert(x['5UTR'], k_length=3), axis=1)

In [45]:
nuclear.head()

,Transcript name,5UTR,3UTR,Mettler C0,Tertile,3UTRK,5UTRK
0,ADC1,ATAAAGGTCGGGCCGGTTGTTTACAATGAAGGGCACAGCCGCGCCA...,AGTGGACTGTCGGCATAAGAATGCTGTGCTAGGAATAGGTACCGTG...,9.527054,2,"[AGT, GTG, TGG, GGA, GAC, ACT, CTG, TGT, GTC, ...","[ATA, TAA, AAA, AAG, AGG, GGT, GTC, TCG, CGG, ..."
1,LHCSR1,AGAGTCATTCCCCAACCCACTTCTCAGCTTGCACCTCCTTGTGCAC...,GTGCCTCTTCCTCGATCGTGATCTGCTTCACGGTTTGGTCGCGGGC...,12.158357,3,"[GTG, TGC, GCC, CCT, CTC, TCT, CTT, TTC, TCC, ...","[AGA, GAG, AGT, GTC, TCA, CAT, ATT, TTC, TCC, ..."
2,GRX2,CACATTGGCACTGGAGTTTATTTGAGGCGATACATGCCTTCTCATG...,GGCGAGGGCGGCGGGGTGGGCAGCAGGTTGTGAGGCGCGTGCAGGG...,14.064211,3,"[GGC, GCG, CGA, GAG, AGG, GGG, GGC, GCG, CGG, ...","[CAC, ACA, CAT, ATT, TTG, TGG, GGC, GCA, CAC, ..."
3,TIM16,GCACCAAACTTTCCCTGGGCCCCCCTCTGGGCGCACTCGAAATGCT...,CGAGGCGTGGCGTGAGGTGACAAGCAGGGGCGATGGATGTTGGGGA...,8.464260,2,"[CGA, GAG, AGG, GGC, GCG, CGT, GTG, TGG, GGC, ...","[GCA, CAC, ACC, CCA, CAA, AAA, AAC, ACT, CTT, ..."
4,Cre01.g056200,ACTTGTCAAAACCTTGCAAGGCTTGCGGCAGGCACTGTCCAGTACC...,TGGGTCAGTGGGGCGTGTGTGTGGCCGGCTGCTGGACCCGTGAGCA...,14.582071,3,"[TGG, GGG, GGT, GTC, TCA, CAG, AGT, GTG, TGG, ...","[ACT, CTT, TTG, TGT, GTC, TCA, CAA, AAA, AAA, ..."


In [46]:
nuclear["Combined"] = nuclear["3UTRK"] + nuclear["5UTRK"]

In [47]:
kmer_sentences = list(nuclear['Combined'])
for e in range(len(kmer_sentences)):
    kmer_sentences[e] = ' '.join(kmer_sentences[e])

In [48]:
print(kmer_sentences[5])

CAG AGG GGC GCG CGC GCA CAC ACA CAG AGG GGC GCG CGA GAG AGG GGT GTT TTG TGC GCA CAG AGG GGA GAG AGC GCA CAA AAT ATC TCC CCT CTT TTA TAG AGA GAG AGA GAT ATG TGC GCG CGC GCC CCC CCA CAA AAC ACT CTG TGC GCT CTT TTA TAG AGG GGC GCT CTG TGC GCG CGG GGG GGC GCC CCA CAA AAG AGC GCA CAG AGC GCT CTG TGC GCA CAG AGT GTC TCC CCA CAT ATT TTC TCG CGT GTG TGA GAA AAA AAT ATA TAT ATT TTA TAC ACA CAA AAG AGC GCC CCC CCT CTG TGC GCT CTT TTG TGT GTG TGC GCA CAG AGC GCA CAG AGG GGC GCC CCG CGC GCA CAA AAC ACT CTG TGA GAG AGC GCG CGC GCT CTG TGA GAC ACA CAA AAC ACT CTG TGC GCC CCA CAT ATA TAG AGT GTT TTT TTT TTG TGT GTT TTG TGA GAA AAC ACA CAC ACC CCT CTG TGC GCT CTC TCT CTT TTT TTT TTT TTC TCC CCG CGG GGG GGC GCA CAC ACG CGG GGG GGC GCT CTC TCC CCG CGA GAC ACG CGG GGA GAA AAT ATC TCC CCA CAC ACG CGG GGA GAG AGA GAT ATA TAC ACG CGG GGC GCA CAT ATG TGT GTA TAC ACT CTT TTT TTG TGG GGA GAA AAA AAT ATG TGC GCC CCA CAA AAA AAT ATA TAG AGA GAG AGT GTG TGA GAT ATC TCC CCT CTG TGC GCG CGA GAC ACG CGG GGG GGC GCG 

In [49]:
cv = CountVectorizer(ngram_range=(8,8))
X = cv.fit_transform(kmer_sentences)
print(X.shape)

(384, 283045)


In [50]:
y = nuclear[['Tertile']]

In [51]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

In [52]:
#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)

In [53]:
from sklearn.naive_bayes import MultinomialNB

nuclearclassifier = MultinomialNB(alpha = 0.0001)
nuclearclassifier.fit(X, np.ravel(y))

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [54]:
from sklearn.ensemble import RandomForestClassifier

nuclearclassifier2 = RandomForestClassifier()
nuclearclassifier2.fit(X, np.ravel(y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [55]:
#y_pred = nuclearclassifier.predict(X_test)

In [56]:
#print(accuracy_score(y_test, y_pred))
#print(precision_score(y_test, y_pred, average='weighted'))
#print(recall_score(y_test, y_pred, average='weighted'))

In [57]:
chloroplast = pd.read_csv("C:/Users/natha/Desktop/MSCi/algae.csv")

In [58]:
chloroplast.head()
chloroplast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene            67 non-null     object 
 1   Average         67 non-null     float64
 2   Normalised avg  67 non-null     float64
 3   3UTR            67 non-null     object 
 4   5UTR            67 non-null     object 
 5   Quintile        67 non-null     int64  
 6   Tertile         67 non-null     int64  
 7   3UTR MFE        67 non-null     float64
 8   3UTRL           67 non-null     int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 4.8+ KB


In [59]:
chloroplast['3UTRK'] = chloroplast.apply(lambda x: kmerconvert(x['3UTR'], k_length=3), axis=1)

In [60]:
chloroplast['5UTRK'] = chloroplast.apply(lambda x: kmerconvert(x['5UTR'], k_length=3), axis=1)

In [61]:
chloroplast["Combined"] = chloroplast["3UTRK"] + chloroplast["5UTRK"]

In [62]:
kmer_sentences2 = list(chloroplast['Combined'])
for e in range(len(kmer_sentences2)):
    kmer_sentences2[e] = ' '.join(kmer_sentences2[e])

In [63]:
print(kmer_sentences2[5])

taa aaa aag agt gtt tta taa aat ata taa aag aga gat att ttt ttt tta taa aat att ttt tta taa aag agt gtt ttc tct cta tat ata tat att tta tat atc tca caa aac act cta tac act ctt tta tac act cta tag agt gtt ttg tga gat ata taa aat ata tat ata tag aga gaa aac act ctt tta taa aaa aat ata tat ata taa aaa aaa aag agt gtt ttt ttt tta taa aat att ttt ttt ttt ttc tct ctt tta taa aat ata tat atg tgt gta tag agt gtt ttg tgg ggc gca cag agt gtt tta tat att ttc tct cta taa aaa aat att tta taa aaa aac act cta tag agg ggt gta taa aaa aaa aag aga gat att ttt ttc tca cac act ctt ttc tct ctt ttt ttt tta tag agc gct cta tag aga gaa aaa aaa aaa aaa aaa aag aga aaa aaa aaa aat atg tgt gta taa aat ata taa aaa aaa aat att ttc tct ctt tta tac acc cca caa aat att tta tat att ttg tgg gga gaa aaa aag aga gat ata tat att tta taa aat ata tat att ttt ttt ttt tta taa aat ata taa aat ata taa aac acc cca cac act cta tat ata taa aaa aat ata taa aat atg tga gac acc cca cag agt gtt tta tat att tta tat atc tct cta tat att 

In [64]:
#cv2 = CountVectorizer(ngram_range=(13,13))
X2 = cv.transform(kmer_sentences2)
print(X2.shape)

(67, 283045)


In [65]:
chloroplast.head()

,Gene,Average,Normalised avg,3UTR,5UTR,Quintile,Tertile,3UTR MFE,3UTRL,3UTRK,5UTRK,Combined
0,atpA,11.88,0.832,tttttaattaagtaggaactcggtatatgctcttttggggtcttat...,attttaatgcttatgctatcttttttatttagtccataaaaccttt...,5,3,-31.3,185,"[ttt, ttt, ttt, tta, taa, aat, att, tta, taa, ...","[att, ttt, ttt, tta, taa, aat, atg, tgc, gct, ...","[ttt, ttt, ttt, tta, taa, aat, att, tta, taa, ..."
1,atpB,11.62,0.814,ttggctctttaagaagaaaacaacttaatggtgtccaaatatttat...,tcatattttaacttattttacttaaattcttacgtataaaccccga...,5,3,-4.9,70,"[ttg, tgg, ggc, gct, ctc, tct, ctt, ttt, tta, ...","[tca, cat, ata, tat, att, ttt, ttt, tta, taa, ...","[ttg, tgg, ggc, gct, ctc, tct, ctt, ttt, tta, ..."
2,atpE,11.36,0.796,ttagatctatgtatttacccaaagagtatactgttcaactctatca...,aataaaaaaaaataaaacttctaaaagcgataaagctagaacattc...,4,3,-30.1,132,"[tta, tag, aga, gat, atc, tct, cta, tat, atg, ...","[aat, ata, taa, aaa, aaa, aaa, aaa, aaa, aaa, ...","[tta, tag, aga, gat, atc, tct, cta, tat, atg, ..."
3,atpF,9.68,0.678,tttttaataaagcttactaacgaattacactaactttactgtattt...,aaataaatttgacttagcttaaatttcagtatatttatatacacaa...,3,2,-230.6,628,"[ttt, ttt, ttt, tta, taa, aat, ata, taa, aaa, ...","[aaa, aat, ata, taa, aaa, aat, att, ttt, ttg, ...","[ttt, ttt, ttt, tta, taa, aat, ata, taa, aaa, ..."
4,atpH,13.85,0.971,ttttaaatttacatgttgtaaaggatatctatagaaatgggagtta...,tcggataaaacatattatgaccgtataatgtttttccaccattgaa...,5,3,-206.4,986,"[ttt, ttt, tta, taa, aaa, aat, att, ttt, tta, ...","[tcg, cgg, gga, gat, ata, taa, aaa, aaa, aac, ...","[ttt, ttt, tta, taa, aaa, aat, att, ttt, tta, ..."


In [66]:
y2 = chloroplast[['Tertile']]

In [67]:
y2_pred = np.empty(67,) 

In [68]:
y2_pred = nuclearclassifier.predict(X2)

In [69]:
print(accuracy_score(y2_pred, y2))
print(precision_score(y2_pred, y2, average='weighted'))
print(recall_score(y2_pred, y2, average='weighted'))

0.4626865671641791
0.6333254675240398
0.4626865671641791


In [70]:
y3_pred = nuclearclassifier2.predict(X2)
print(accuracy_score(y3_pred, y2))
print(precision_score(y3_pred, y2, average='weighted'))
print(recall_score(y3_pred, y2, average='weighted'))

0.34328358208955223
0.9701492537313433
0.34328358208955223


C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
